## Доразметка базового датасета

Хочу с помощью LLM извлечь пространственные признаки и получить датасет вида

```
{"scene": 
    {"location": "склад", 
     "objects": [{"коробка": []}, 
                 {"паллета": ["деревянная"]}, 
                 {"тележка": ["металлическая", "квадратная", "тяжелая"]}, 
                 {"стеллаж": ["высокий"]}],
     "relations":[("объект1", "<связь>", "объект2"), ("объект1", "<связь>", "объект3") ...]
     }
}
```

то есть с помощью LLM добавляем к исходному датасету связи

```
"relations":[("объект1", "<связь>", "объект2"), ("объект1", "<связь>", "объект3") ...]
```

то есть по сути у нас все уже выделено - наша миссия только довыделить пространственные связи

В целом модель отрабатывает хорошо, но иногда (редко) выползают глагольные связки не имеющие отношения к пространственному расположению например **('фонарь', 'освещает', 'скамейка')**. Будем делать пост-фильтрацию с помощью spacy

In [3]:
import sys
import os
import random
import logging
import json
import copy

from glob import glob
from tqdm import tqdm
from dotenv import load_dotenv
from pathlib import Path
from collections import defaultdict, Counter

lib_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(lib_path)

from library.locations import LOCATIONS
from library.llm_connector import request_llm
from library.graph_vizualization import scene_to_graph_sp, draw_scene_graph_sp

import spacy
nlp = spacy.load("ru_core_news_sm")

dotenv_path = Path(os.getcwd()).resolve().parent / '.env'
load_dotenv(dotenv_path)

LOG_FILE = "ds_generation.log"
logging.basicConfig(filename=LOG_FILE, level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

API_KEY = os.getenv("API_KEY")
API_KEY_DS = os.getenv("API_KEY_DS")

#input_dir = "dataset_syntetic_v4"  
#output_dir = "dataset_syntetic_v5_spacial"  

input_dir = "dataset_validation"  
output_dir = "dataset_validation_spacial"  



In [5]:
def extract_object_names(object_list):
    return [list(obj.keys())[0] for obj in object_list]

# Function to build prompt for a single record
def build_prompt(description, object_names):
    obj_list_str = ", ".join(f'"{name}"' for name in object_names)
    prompt = f"""
Ты анализируешь описание сцены и выделяешь пространственные связи между следующими объектами:
Объекты: [{obj_list_str}]

Описание: "{description}"

Верни все строго только пространственные связи в формате:
[("объект1", "предлог", "объект2")]

Где "объект1" и "объект2" — из списка объектов, а "предлог" — пространственное отношение 
(например, "на", "рядом с", "возле", "перед", "под" и т.д.). Если связей нет, верни пустой список: []

Связи должны быть направленными: если один объект находится **на** другом, то в тройке 
**сначала тот, кто сверху**, затем — тот, кто снизу. Например: ("тарелка", "на", "стул"), а не наоборот.

То же касается всех пространственных предлогов: сначала объект, чьё положение 
определяется (который "лежит", "стоит" и т.д.), потом — опорный объект (относительно которого он размещён).
"""
    return prompt

def is_valid_relation(relation_str):
    relation_str = relation_str.lower().strip()
    doc = nlp(relation_str)

    # Если в выражении нет предлога — баним
    has_adp = any(tok.pos_ == "ADP" for tok in doc)  # ADP = adposition (предлоги)
    if not has_adp:
        return False
    return True

def parse_relations(llm_output, object_names):
    try:
        if not llm_output.strip().startswith("["):
            return None
        parsed = eval(llm_output, {"__builtins__": {}})
        if not isinstance(parsed, list):
            return None
        validated = []
        for item in parsed:
            if (isinstance(item, tuple) and len(item) == 3 and 
                item[0] in object_names and item[2] in object_names and 
                isinstance(item[1], str) and is_valid_relation(item[1])):
                validated.append(item)
        return validated
    except Exception:
        return None
    
    
# Process all batches
input_files = sorted(glob(os.path.join(input_dir, "dataset_batch_*.jsonl")))

for file_path in tqdm(input_files, desc="Processing batches"):
    output_lines = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            record = json.loads(line)
            scene = record["scene"]
            description = record["description"]
            object_names = extract_object_names(scene["objects"])

            prompt = build_prompt(description, object_names)
            llm_output = request_llm(prompt, API_KEY, temperature=0)

            relations = parse_relations(llm_output, object_names)
            if relations is None:
                relations = []

            scene["relations"] = relations
            updated_record = {
                "scene": scene,
                "description": description
            }
            #print(updated_record)
            output_lines.append(json.dumps(updated_record, ensure_ascii=False))

    output_filename = os.path.basename(file_path).replace("dataset_batch", "dataset_spacial_batch")
    output_path = os.path.join(output_dir, output_filename)
    with open(output_path, 'w', encoding='utf-8') as f_out:
        for line in output_lines:
            f_out.write(line + "\n")

Processing batches: 100%|█████████████████████████| 5/5 [05:55<00:00, 71.08s/it]
